<center>
<h1> Recherche Opérationnelle : TP 3 et 4 </h1>
<h1> Année 2020-2021 - 2e année département Sciences du Numérique </h1>
<h1> Hamza Mouddene </h1>
<h1> Anass Tyoubi </h1>    
</center>

## 1 Programmation dynamique avec Bellman-Ford
***

### 1.1 Calcul du plus court chemin entre deux sommets d’un graphe
***

Nous allons implémenter en Julia l’algorithme de Bellman-Ford permettant de calculer le plus court chemin
entre un sommet source s et tous les autres sommets d’un graphe quelconque.

#### Modélisation du problème

In [1]:
#importer les packages utiles, le manager de package Pkg etant un package
import Pkg; Pkg.add("Cbc")
Pkg.add("JuMP")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100,0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [221]:
function getIndex(vertex, n, item)
    for i in 1 : n
        if vertex[i] == item
            return i
        end
    end
end

function bellman_ford(n, m, graph, vertex, source)
    
    distance = zeros(n)
    
    for i in 1 : n
        distance[i] = typemax(Int64)
        previous[i] = 'A'
    end
    
    distance[source] = 0
    
    for i in 1 : n
        for j in 1 : m
            u = graph[j][1]
            v = graph[j][2]
            w = graph[j][3]

            if (distance[getIndex(vertex, n, u)] + w < distance[getIndex(vertex, n, v)])
                distance[getIndex(vertex, n, v)] = distance[getIndex(vertex, n, u)] + w
                previous[getIndex(vertex, n, v)] = u
            end
        end
    end
    
    for i in 1 : m
        u = graph[i][1]
        v = graph[i][2]
        w = graph[i][3]
        if (distance[getIndex(vertex, n, u)] != typemax(Int64) 
            && distance[getIndex(vertex, n, u)] + w  < distance[getIndex(vertex, n, v)])  
            println("Graph contains negative weight cycle")
        end
    end
    
    println("distance = ", distance)
    println("previous = ", previous)
    
    print("Le plus court chemin : ")
    L = [vertex[n]]
    j = n
    while j != source
        append!(L, [vertex[getIndex(vertex, n, previous[j])]])
        j = getIndex(vertex, n, previous[j])
    end
    
    p = size(L, 1)
    
    print(L[p])
    for j in 1 : p - 1
        print("-->", L[ p - j])
    end
    println("")
end

bellman_ford (generic function with 1 method)

#### Test avec l'exemple du sujet

In [222]:
vertex = ['A', 'B', 'C', 'D', 'E', 'F']
graph = [
    ('A', 'B', 3), ('A', 'E', 5),
    ('B', 'C', 4),
    ('C', 'D', 2),
    ('D', 'F', 3),
    ('E', 'B', -1), ('E', 'D', 9)
    ] 
source = 1
previous = ['A', 'A', 'A', 'A', 'A', 'A']
bellman_ford(length(vertex), length(graph), graph, vertex, source)

distance = [0.0, 3.0, 7.0, 9.0, 5.0, 12.0]
previous = ['A', 'A', 'B', 'C', 'A', 'D']
Le plus court chemin : A-->B-->C-->D-->F


### 1.2 Calcul du plus long chemin entre deux sommets d’un graphe
***

Pour trouver le plus long chemin avec au maximum n−1 arètes entre deux sommets d'un graphe, il faut appliquer des modifications à la relation de récurrence et à l’algorithme de programmation dynamique de Bellman-Ford.

Une solution simple consiste à convertir le graph G à -G, c'est à dire intervertir le signe des poids de chaque arc dans le graphe G, puis calculer le plus court chemin en passant par l'algorithme implémenté dans la question précedente en respectant bien sur la condition que le chemin ne doit pas dépasser n-1 arètes.

#### Modélisation du problème

In [223]:
function bellman_ford_v1(n, m, graph, vertex, source)
    
    distance = zeros(n)
    
    for i in 1 : n
        distance[i] = typemax(Int64)
        previous[i] = 'A'
    end
    
    distance[source] = 0
    
    for i in 1 : n
        for j in 1 : m
            u = graph[j][1]
            v = graph[j][2]
            w = -graph[j][3]

            if (distance[getIndex(vertex, n, u)] + w < distance[getIndex(vertex, n, v)])
                distance[getIndex(vertex, n, v)] = distance[getIndex(vertex, n, u)] + w
                previous[getIndex(vertex, n, v)] = u
            end
        end
    end
    
    for i in 1 : m
        u = graph[i][1]
        v = graph[i][2]
        w = -graph[i][3]
        if (distance[getIndex(vertex, n, u)] != typemax(Int64) 
            && distance[getIndex(vertex, n, u)] + w  < distance[getIndex(vertex, n, v)])  
            println("Graph contains negative weight cycle")
        end
    end
    println("distance = ", abs.(distance))
    println("previous = ", previous)
    
    print("Le plus long chemin : ")
    L = [vertex[n]]
    j = n
    while j != source
        append!(L, [vertex[getIndex(vertex, n, previous[j])]])
        j = getIndex(vertex, n, previous[j])
    end
    
    p = size(L, 1)
    
    print(L[p])
    for j in 1 : p - 1
        print("-->", L[ p - j])
    end
    println("")

end

bellman_ford_v1 (generic function with 1 method)

#### Test avec l'exemple du sujet

In [224]:
bellman_ford_v1(length(vertex), length(graph), graph, vertex, source)

distance = [0.0, 4.0, 8.0, 14.0, 5.0, 17.0]
previous = ['A', 'E', 'B', 'E', 'A', 'D']
Le plus long chemin : A-->E-->D-->F


## 2 Extensions et adaptations
***

### 2.1 Construction d’un réseau de transmission à vitesse maximale
***

La société **ALPHA** désire relier par un réseau, un processeur central *P* à divers processeurs (éventuellement par des intermédiaires). Une vitesse de transmission *lij* en bauds est associée à chaque
liaison possible (bidirectionnelle) reliant le processeur *i* au processeur *j*. La vitesse de transmission
d’un chemin du processeur central *P* au processeur *K* est égale au plus petit des *lij* rencontrés sur
ce chemin. **ALPHA** souhaite trouver les chemins de vitesses maximales reliant le processeur central
*P* à tous les autres processeurs *K* du réseau.

Cela nécessite une modification au niveau de la relation de récurrence et à l’algorithme de **BellmanFord** pour calculer le chemin de vitesse maximale entre deux sommets d’un graphe.

#### Modélisation du problème

In [225]:
function bellman_ford_v2(n, m, graph, vertex, source)
    maxi = zeros(n)
    distance = zeros(n)
    
    for i in 1 : n    
        maxi[i] = -typemax(Int64)
        distance[i] = -typemax(Int64)
    end
    
    distance[source] = typemax(Int64)
    
    for i in 1 : n
        for j in 1 : m
            u = graph[j][1]
            v = graph[j][2]
            w = graph[j][3]

            if (min(distance[getIndex(vertex, n, u)], w) > maxi[getIndex(vertex, n, v)])
                distance[getIndex(vertex, n, v)] = min(distance[getIndex(vertex, n, u)], w)
                maxi[getIndex(vertex, n, v)] = min(distance[getIndex(vertex, n, u)], w)
            end
        end
    end
    
    println("distance = ", maxi)
    
    for i in 1 : n
        println("P -> ", vertex[i], " avec une vitesse de ", maxi[i])
    end 
end

bellman_ford_v2 (generic function with 1 method)

#### Test avec l'exemple du sujet

In [226]:
vertex = ['P', '1', '2', '3', '4', '5', '6']
graph = [
    ('P', '1', 5), ('P', '4', 1), ('P', '6', 3),
    ('1', 'P', 5), ('1', '2', 4),
    ('2', '1', 4), ('2', '3', 1), ('2', '4', 2), ('2', '5', 2),
    ('3', '2', 1), ('3', '5', 2),
    ('4', 'P', 1), ('4', '2', 2), ('4', '5', 3), ('4', '6', 5),
    ('5', '2', 2), ('5', '3', 2), ('5', '4', 3),
    ('6', 'P', 3), ('6', '4', 5)
    ] 
source = 1
bellman_ford_v2(length(vertex), length(graph), graph, vertex, source)

distance = [5.0, 5.0, 4.0, 2.0, 3.0, 3.0, 3.0]
P -> P avec une vitesse de 5.0
P -> 1 avec une vitesse de 5.0
P -> 2 avec une vitesse de 4.0
P -> 3 avec une vitesse de 2.0
P -> 4 avec une vitesse de 3.0
P -> 5 avec une vitesse de 3.0
P -> 6 avec une vitesse de 3.0


### 2.2 Fiabilité de procédé de fabrication de semi-conducteurs
***

L’entreprise de semi-conducteurs **GAMMA** souhaite déterminer le procédé de fabrication le plus
sûr pour leurs nouveaux processeurs **SX-42**. Il y a en effet plusieurs façons de faire pour transformer
la matière première jusqu’au produit fini (enchainement d’opérations de gravure, de vernissage, de
nettoyage, de dopage, etc).
Le processus de fabrication de processeurs est extrêmement sensible et si à une étape il y a le moindre problème alors la plaque de silicium est perdue et détruite.

Ci-dessus le schéma descriptif des processus de fabrication possibles. Les produits semi-finis
intermédiaires sont désignés par les abréviations *S1, S2, S3, A1, A2, A3, A4, D1, D2, FG, FC*.
La probabilité de succès est indiquée pour chaque étape possible. Par exemple si l’on dispose de
produits semi-finis de type *S2*, alors on a le choix entre deux processus. Le premier donnera des
produits semi-finis de type *A2*, avec une probabilité de succès à *4s* de *0.97* (il y aura donc *3%*
déchets). Le second  processus fournira des produits semi-finis de type *A3* avec une probabilité sur
le période, avec une probabilité de succès de *0.98* (il y aura donc *2%* de déchets). La problématique
est de déterminer le procédé de fabrication le plus sûr, c-à-d comportant moins de déchets.

(a) En utilisant le fait que pour tout $a, b ∈ R+$ on a $log ab = log a + log b$, montrer que l’on
peut transformer la recherche du procédé de fabrication le plus sûr en un problème de plus
long chemin dans un graphe à préciser. Implémenter et tester sur l’exemple fourni puis avec
différents jeux de données de votre choix.

#### Modélisation du problème

In [254]:
function bellman_ford_v3(n, m, graph, vertex, source)
    
    distance = zeros(n)
    previous = string.(zeros(n))
    
    for i in 1 : n
        distance[i] = Inf
        previous[i] = "Matières premières"
    end
    
    distance[source] = 0
    
    for i in 1 : n
        for j in 1 : m
            u = graph[j][1]
            v = graph[j][2]
            w = -log(graph[j][3])
            
            if (distance[getIndex(vertex, n, u)] + w < distance[getIndex(vertex, n, v)])
                distance[getIndex(vertex, n, v)] = distance[getIndex(vertex, n, u)] + w
                previous[getIndex(vertex, n, v)] = u
            end
        end
    end
    
    
    println("Le procédé de fabrication le plus sûr est : ")
    L = [vertex[n]]
    j = n
    while j != source
        append!(L, [vertex[getIndex(vertex, n, previous[j])]])
        j = getIndex(vertex, n, previous[j])
    end
    
    p = size(L, 1)
    
    print(L[p])
    for j in 1 : p - 1
        print(" --> ", L[ p - j])
    end
    println("\nAvec une probabilité de succés de ", exp(-distance[n]))
end

bellman_ford_v3 (generic function with 1 method)

#### Test avec l'exemple du sujet

In [255]:
vertex = ["Matières premières", "S1", "S2", "S3", "A1", "A2", "A3", "A4", "D1", "D2", "FG", "FX", "Produit fini"]
graph = [
    ("Matières premières", "S1", 0.98), ("Matières premières", "S2", 0.97), ("Matières premières", "S3", 0.99),
    ("S1", "A1", 0.96), ("S1", "A2", 0.95),
    ("S2", "A2", 0.97), ("S1", "A3", 0.98),
    ("S3", "A3", 0.95), ("S1", "A4", 0.93),
    ("A1", "D1", 0.99),
    ("A2", "D1", 0.96),
    ("A3", "D1", 0.97), ("A3", "D2", 0.98),
    ("A4", "D2", 0.99),
    ("D1", "FG", 0.98), ("D1", "FX", 0.99),
    ("D2", "FG", 0.96), ("D2", "FX", 0.99),
    ("FG", "Produit fini", 0.99),
    ("FX", "Produit fini", 0.93)
    ] 
source = 1
bellman_ford_v3(length(vertex), length(graph), graph, vertex, source)

Le procédé de fabrication le plus sûr est : 
Matières premières --> S1 --> A3 --> D1 --> FG --> Produit fini
Avec une probabilité de succés de 0.9038266775999999


(b) On peut aussi directement modifier l’algorithme de **Bellman-Ford** vu dans ce TP pour l’adapter à la recherche d’un plus sûr chemin, c-à-d en un chemin $x1,...,xk$ tel que le produit $Qk−1 i=1 p(xi, xi+1)$ soit maximum. Implémenter et tester sur l’exemple fourni puis avec différents jeux de données de votre choix, et comparer au résultat précédent.

#### Modélisation du problème

In [252]:
function bellman_ford_v4(n, m, graph, vertex, source)
    distance = zeros(n)
    previous = string.(zeros(n))
    
    for i in 1 : n
        distance[i] = -Inf
        previous[i] = "Matières premières"
    end
    
    distance[source] = 1
    
    for i in 1 : n
        for j in 1 : m
            u = graph[j][1]
            v = graph[j][2]
            w = graph[j][3]
            
            if (distance[getIndex(vertex, n, u)] * w > distance[getIndex(vertex, n, v)])
                distance[getIndex(vertex, n, v)] = distance[getIndex(vertex, n, u)] * w
                previous[getIndex(vertex, n, v)] = u
            end
        end
    end
    
    
    println("Le procédé de fabrication le plus sûr est : ")
    L = [vertex[n]]
    j = n
    while j != source
        append!(L, [vertex[getIndex(vertex, n, previous[j])]])
        j = getIndex(vertex, n, previous[j])
    end
    
    p = size(L, 1)
    
    print(L[p])
    for j in 1 : p - 1
        print(" --> ", L[ p - j])
    end
    println("\nAvec une probabilité de succés de ", distance[n])
end

bellman_ford_v4 (generic function with 1 method)

#### Test avec l'exemple du sujet

In [253]:
bellman_ford_v4(length(vertex), length(graph), graph, vertex, source)

Le procédé de fabrication le plus sûr est : 
Matières premières --> S1 --> A3 --> D1 --> FG --> Produit fini
Avec une probabilité de succés de 0.9038266775999999
